In [1]:
import numpy as np 
from functools import reduce

In [2]:
def xor(a, b):
    return np.logical_xor(a, b, dtype='uint8').astype("uint8")

def and_(a, b):
    return np.logical_and(a, b, dtype='uint8').astype("uint8")


def strip_zeros(a):
    return np.trim_zeros(a, trim='b')

In [3]:
def gf2_add(a, b):  
    a, b = strip_zeros(a), strip_zeros(b)

    N = len(a)
    D = len(b)   
    F = min(N,D)

    if N == D:
        res = xor(a, b)
    
    elif N>D:
        res = np.concatenate((xor(a[:F], b), a[F:]))
    else:       
        res = np.concatenate((xor(b[:F], a), b[F:]))

    return strip_zeros(res)

In [4]:
a = np.array([1,0,1], dtype="uint8")
b = np.array([1,1], dtype="uint8")
gf2_add(a,b)

array([0, 1, 1], dtype=uint8)

In [5]:
def gf2_mul(a, b):
    m=np.convolve(a,b)
    return strip_zeros(np.mod(m,2))

In [6]:
a = np.array([1,0,1], dtype="uint8")
b = np.array([1,1,1], dtype="uint8")
gf2_mul(a,b)

array([1, 1, 0, 1, 1], dtype=uint8)

In [7]:
def gf2_div(a, b):
    a, b = strip_zeros(a), strip_zeros(b)

    if not b.any(): 
        return "error"
    elif len(b) > len(a):
        q = np.array([])
        return q, a

    else:
        u = a.astype("uint8")
        v = b.astype("uint8")

        m = len(u) - 1
        n = len(v) - 1
        q = np.zeros((max(m - n + 1, 1),), "uint8")
        r = u.astype("uint8")

        for k in range(0, m - n + 1):
            d = r[m - k].astype("uint8")
            q[-1 - k] = d
            r[m - k - n:m - k + 1] = xor(r[m - k - n:m - k + 1], and_(d, v))

        r = strip_zeros(r)

    return q, r

In [16]:
x = np.array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], dtype="uint8")
y = np.array([1, 0, 0, 0, 1], dtype="uint8")

a,b = gf2_div(x, y)
gf2_add(gf2_mul(a,y),b) == x

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [18]:
b

array([1, 0, 1, 1], dtype=uint8)

In [9]:
def GF2_polynomial_sum(poly1, poly2):
    l1 = len(poly1)
    l2 = len(poly2)

    if l1 < l2:
        poly1 = '0'*(l2-l1) + poly1
    elif l2 < l1:
        poly2 = '0'*(l1-l2) + poly2
    else:
        pass

    sum_list = [((int(x) + int(y))%2) for x,y in zip(list(poly1),list(poly2))]
    leading_zeroes = True
    while leading_zeroes == True:
        if len(sum_list) == 1:
            leading_zeroes = False
        elif sum_list[0] == 0:
            del sum_list[0]
        else:
            leading_zeroes = False
    sum = ''
    for x in sum_list:
        sum += str(x)
    return sum

In [10]:
# Polynomial multiplication in GF(2)
def GF2_polynomial_product(poly1, poly2):
    l1 = len(poly1)
    l2 = len(poly2)

    l = l1+l2-1
    poly1 = list(poly1)
    poly1.reverse()

    poly2 = list(poly2)
    poly2.reverse()
    product_list = [0]*l
    for i in range(0,l1):
        for j in range(0,l2):
            c = int(poly1[i])*int(poly2[j])
            product_list[i+j] = ((product_list[i+j] + c) % 2)

    product_list.reverse()
    product = ''
    for x in product_list:
        product += str(x)

    return product

In [11]:
def GF2_polynomial_remainder(dividend,divisor):

    if divisor == '0':
        print("division by 0")
        return None
    if divisor == '1':
        return '0'

    while len(dividend) >= len(divisor):

        deg_dividend = len(dividend)
        deg_divisor = len(divisor)
        shift = deg_dividend - deg_divisor

        # Calculate the shift of the divisor.
        shifted_factor = '1' + '0'*shift
        shifted_divisor = GF2_polynomial_product(shifted_factor,divisor)

        dividend = GF2_polynomial_sum(dividend, shifted_divisor)

    return dividend

In [12]:
def GF2_polynomial_quotient(dividend,divisor):
    quotient = ['0']*((len(dividend)-len(divisor))+1)
    if divisor == '0':
        print("division by 0")
        return None

    if divisor == '1':
        return dividend

    while len(dividend) >= len(divisor):
        deg_dividend = len(dividend)
        deg_divisor = len(divisor)
        shift = deg_dividend - deg_divisor
        quotient[(len(quotient) - (shift+1))] = '1'

        shifted_factor = '1' + '0'*shift
        shifted_divisor = GF2_polynomial_product(shifted_factor,divisor)
      
        dividend = GF2_polynomial_sum(dividend, shifted_divisor)
        
    quotient_str = ''
    for x in quotient:
        quotient_str += str(x)

    return quotient_str

In [13]:
a = '101'
b = '111'
GF2_polynomial_sum(a,b)

'10'

In [14]:
a = '101'
b = '111'
GF2_polynomial_product(a,b)

'11011'

In [24]:
a='1100101'
b='1101'
print(GF2_polynomial_quotient(a,b))
print(GF2_polynomial_remainder(a,b))


1001
0


In [ ]:
#!/bin/python3

import numpy as np


def number_to_binary_array(n, r):
    """
    input integer
    output binary representation of integer as values in an array that can hold numbers less than 2^r
    """
    res = [int(i) for i in bin(n)[2:]]
    while len(res) < r:
        res = [0] + res
    return res


def make_identity_matrix(n):
    """
    input n
    output list of lists, identity matrix I_n
    """
    return [number_to_binary_array(2**i, n) for i in reversed(range(n))]


def make_extended_Golay_generator():
    """
    returns the generator for the extended Golay code
    """
    B = [[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0]]
    for _ in range(10):
        next_line = B[-1][:]
        next_line = next_line + [next_line.pop(0)]
        B.append(next_line)
    j = [[1]*11]
    G_extended = np.concatenate((np.array(B), np.array(j)))
    j_0 = [[1]]*11 + [[0]]
    G_extended = np.concatenate((np.array(G_extended), np.array(j_0)), axis=1)
    Id_12 = make_identity_matrix(12)
    G_extended = np.concatenate((Id_12, np.array(G_extended)), axis=1)
    return G_extended


G_extended = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
                       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
                       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
                       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
                       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
                       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
                       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
                       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
                       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1],
                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])

G = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
              [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


def encode_extended_Golay(info):
    global G_extended

    if type(info) == str:
        info = [int(i) for i in info]
        res = info @ G_extended % 2
        return ''.join(str(i) for i in res)

    if type(info) == list:
        return list(info @ G_extended % 2)

    if type(info) == np.ndarray:
        return info @ G_extended % 2

def encode_Golay(info):
    global G

    if type(info) == str:
        info = [int(i) for i in info]
        res = info @ G % 2
        return ''.join(str(i) for i in res)

    if type(info) == list:
        return list(info @ G % 2)

    if type(info) == np.ndarray:
        return info @ G % 2

#encode_Golay('010101010101'),'01010101010110110100001'
#encode_extended_Golay('101010101010'), '01010101010110110100001'